<a href="https://colab.research.google.com/github/seiji0203/BERT/blob/master/BERT__%E5%9F%BA%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第4章　Hugging Transformer

In [ ]:
# 4-1 使用するライブラリのinstall
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

     |████████████████████████████████| 2.1 MB 4.2 MB/s 
     |████████████████████████████████| 486 kB 48.5 MB/s 
     |████████████████████████████████| 13.4 MB 215 kB/s 
     |████████████████████████████████| 3.3 MB 30.2 MB/s 
     |████████████████████████████████| 895 kB 49.2 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=ed1d7221bb12b367f59f18bbe0568630d654d336c031e9da66d5fd08919a1daa
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [ ]:
# 4-2　ライブラリのinstall
import torch
from transformers import BertJapaneseTokenizer, BertModel

In [ ]:
# 4-3 学習済みトークナイザをロード
# MeCabを用いて単語を分割、、WordPieceを用いて単語をトークンに分割する
# 日本語モデルでは語彙はは32000のトークンを含んでいる
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

In [ ]:
# トークン化
tokenizer.tokenize('今日は自然言語処理の勉強をしてみよう。')

['今日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'し', 'て', 'みよ', 'う', '。']

In [ ]:
# サブワードに分解では##が付く
tokenizer.tokenize('明日はマシンラーニングの勉強をしよう。')

['明日', 'は', 'マシン', '##ラー', '##ニング', 'の', '勉強', 'を', 'しよ', 'う', '。']

In [ ]:
# 未知語を表す特殊トークン特殊トークン[UNK]
tokenizer.tokenize('機械学習を中国語にすると机器学习だ。')

['機械', '学習', 'を', '中国', '語', 'に', 'する', 'と', '机', '器', '学', '[UNK]', 'だ', '。']

In [ ]:
# 符号化
# 4-7
input_ids = tokenizer.encode('明日は自然言語処理の勉強をしよう。')
print(input_ids)

[2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 205, 8, 3]


In [ ]:
# 4-8 関数の関数のencode()はデフォルトでトークン列の先頭に特殊トークン[CLS]、末尾に[SEP]が足されるようになっている。
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。', '[SEP]']

In [ ]:
# 4-9 特殊トークン特殊トークン[PAD]は語彙が不足している場合に末尾に追記されるされる
text = '明日の天気は晴れだ。'
encoding = tokenizer(text, max_length=12, padding='max_length', truncation=True) # padding='max_length', truncation=Trueを指定することでIDがmax_length=12に調整される
print('# encoding:')
print(encoding)

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokenes:')
print(tokens)

# encoding:
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}
# tokenes:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


In [ ]:
# 4-10 max_lengthを文字数より少なくしていすると末尾のいくつかのトークンは取り除かれる
encoding = tokenizer(text, max_length=6, padding='max_length', truncation=True)
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print(tokens)

['[CLS]', '明日', 'の', '天気', 'は', '[SEP]']


In [ ]:
# 4-11 入力として複数の文章を入れることで複数の文章をまとめてできる
text_list = ['明日の天気は晴れだ。', 'パソコンが急に動かなくなった。']
tokenizer(text_list, max_length=10, padding='max_length', truncation=True)

{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
# 4-12 文書の中で最長の文に揃えたいときは揃えたいときはpadding=longestにする
tokenizer(text_list, padding='longest')

{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 8, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
# 4-13 符号化するときにreturn_tensors='pt'引数としてを加えることで数値配列がテンソルとして出力され、BERTにそのまま入力できる
tokenizer(
    text_list,
    max_length=10,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)
# tensor()はtorch.Tensorであることを示している

{'input_ids': tensor([[    2, 11475,     5, 11385,     9, 16577,    75,     8,     3,     0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# 符号化されたデータをBERTに入力し、それぞれのトークンに対する出力を得る方法について
# 4-14 モデルのロード
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
bert = BertModel.from_pretrained(model_name)

# BERTををGPUに載せる載せる
bert = bert.cuda()

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:
print(bert.config)
# hidden_size": 768, BERTの出力は出力は768次元
# max_position_embeddings": 512, 最大で出力出来るトークン列はトークン列は512
# num_hidden_layers": 12,　レイヤー数はレイヤー数は12
# モデルのパラメータ数はパラメータ数は1億1千万程度

BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



In [ ]:
# バッチサイズ（まとめて処理する文章の数（今回は2））
# 4-16
text_list = ['明日は自然言語処理の勉強をしよう。',
             '明日はマシンラーニングの勉強をしよう。'
             ]
# 文章の符号化
encoding = tokenizer(
    text_list,
    max_length=32,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# データをデータをGPUに載せる載せる
encoding = { k:v.cuda() for k, v in encoding.items()}

# BERTでの処理
# bert(**encoding)　＝　input_ids, attention_mask, token_type_ids のencodingを一括でできる
output = bert(**encoding) # それぞれの入力は二次元の二次元のtorch.Tensor
last_hidden_state = output.last_hidden_state # 最終層の出力

In [ ]:
print(last_hidden_state.size()) # テンソルのサイズ

torch.Size([2, 32, 768])


# 第５章　文章の穴埋め

In [ ]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

In [ ]:
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

In [ ]:
# トークナイザとモデルのロード
# 5-3
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
# BERTをGPUに載せる
bert_mlm = bert_mlm.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# トークン化
# 5-4
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


In [ ]:
# input_ids = tokenizer.encode('今日は[MASK]へ行く。')
# print(input_ids)

In [ ]:
# 特殊トークン[MASK]に入るトークンを語彙から予測する
# 5-5
# 文章を符号化（tensor型）してGPUに載せる
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()

# BERTに入力し、分類スコアを得る
# 系列長を揃える必要がないので、単にinput_idsのみを入力する
with torch.no_grad():
  output = bert_mlm(input_ids=input_ids)
  scores = output.logits

In [ ]:
# 5-6
# ID列で'[MASK]'(IDは4)の位置を調べる
mask_position = input_ids[0].tolist().index(4)

# スコアが最も良いトークンのIDを取り出し、トークンに変換する
id_best = scores[0, mask_position].argmax(-1).item()
token_best = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace('##', '')

# [MASK]を上で求めたトークンに置き換える
text = text.replace('[MASK]', token_best)

print(text)


今日は東京へ行く。


In [ ]:
# 5-7
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
    """
    文章中の最初の[MASK]をスコアの上位のトークンに置き換える。
    上位何位まで使うかは、num_topkで指定。
    出力は穴埋めされた文章のリストと、置き換えられたトークンのスコアのリスト。
    """
    # 文章を符号化し、BERTで分類スコアを得る。
    input_ids = tokenizer.encode(text, return_tensors='pt')
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits
    
    # スコアが上位のトークンとスコアを求める。
    mask_position = input_ids[0].tolist().index(4) 
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices # トークンのID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk) # トークン
    scores_topk = topk.values.cpu().numpy() # スコア

    # 文章中の[MASK]を上で求めたトークンで置き換える。
    text_topk = [] # 穴埋めされたテキストを追加する。
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(text.replace('[MASK]', token, 1))

    return text_topk, scores_topk

text = '今日は[MASK]へ行く。'
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm, 20)
print(*text_topk, sep='\n')

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。
今日はパリへ行く。
今日は日本へ行く。
今日はラスベガスへ行く。
今日は北海道へ行く。
今日は刑務所へ行く。
今日は大学へ行く。
今日は海へ行く。
今日はジムへ行く。
今日はここへ行く。
今日はロサンゼルスへ行く。


In [ ]:
# 貪欲法
# [MASK][MASK]場合場合32,000^2通りの可能性があるため近似する
# 最初の[MASK]をまず埋める、ついで次の[MASK]を埋める。
def greedy_prediction(text, tokenizer, bert_mlm):
  """
  [MASK]を含む文章の入力として、貪欲法で穴埋めを行なった文章を出力する
  """
  # 前から順に[MASK]を一つづつ、スコアの最も高いトークンに置き換える
  for _ in range(text.count('[MASK]')):
    text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
  return text

text = '今日は[MASK][MASK]へ行く。'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は、東京へ行く。'

In [ ]:
def greedy_prediction(text, tokenizer, bert_mlm):

  for _ in range(text.count('[MASK]')):
    text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
  return text

text = '今日は[MASK][MASK][MASK][MASK][MASK][MASK]。'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は社会社会的地位が高い。'

In [ ]:
# ビームサーチ（文章生成系：複数の文書を出力）
def beam_search(text, tokenizer, bert_mlm, num_topk):
    """
    ビームサーチで文章の穴埋めを行う。
    """
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for _ in range(num_mask):
        # 現在得られている、それぞれの文章に対して、
        # 最初の[MASK]をスコアが上位のトークンで穴埋めする。
        text_candidates = [] # それぞれの文章を穴埋めした結果を追加する。
        score_candidates = [] # 穴埋めに使ったトークンのスコアを追加する。
        for text_mask, score in zip(text_topk, scores_topk):
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask, tokenizer, bert_mlm, num_topk
            )
            text_candidates.extend(text_topk_inner)
            score_candidates.append( score + scores_topk_inner )

        # 穴埋めにより生成された文章の中から合計スコアの高いものを選ぶ。
        score_candidates = np.hstack(score_candidates)
        idx_list = score_candidates.argsort()[::-1][:num_topk]
        text_topk = [ text_candidates[idx] for idx in idx_list ]
        scores_topk = score_candidates[idx_list]

    return text_topk

text = "今日は[MASK][MASK]へ行く。"
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日はお台場へ行く。
今日はお祭りへ行く。
今日はゲームセンターへ行く。
今日はお風呂へ行く。
今日はゲームショップへ行く。
今日は東京ディズニーランドへ行く。
今日はお店へ行く。
今日は同じ場所へ行く。
今日はあの場所へ行く。
今日は同じ学校へ行く。


In [ ]:
text = "今日は[MASK][MASK][MASK][MASK][MASK]"
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は社会社会学会所属。
今日は社会社会学会会長。
今日は社会社会に属する。
今日は時代社会に属する。
今日は社会社会学会理事。
今日は時代社会にあたる。
今日は社会社会にある。
今日は社会社会学会会員。
今日は時代社会にある。
今日は社会社会になる。
